# First eval

This jupyter notebook is for trying out what research I have read and my ideas in this [Notion](https://www.notion.so/Model-evaluation-170d3cfe027d80889de3cfbb35b531a4).

I played around with a Mistral Large model to build a mathematics chatbot. It is simple and I prompt engineered it. I might change the model to a smaller and cheaper one, but I still want to use a model that is considerd good enough. My plan is to use three small open source models to evaluate the mahtematics model. I'll also use different models for the mathematics chatbot and for the evaluation models to get realible results.

I'll use this [dataset](https://github.com/google-deepmind/mathematics_dataset?tab=readme-ov-file) as a test data set. I'll select 50 mathematics questiosn from the dataset. Then I'll ask the mathematicsbot the same questions. 
Then I'll let: 
- First a Mistral Large model compare the answers. Same model as in the mathematicsbot but different prompt engineering. 
    - This is not a good way to evaluate models, but I wish to try it. Espeacilly when it is only 50 questions it is manageble for me to compare the answers.
- Then I'll let three smaller open-source models compare the answers.
- After that I'll add a fourth model to compare the answers of the three open-open source model to give a final evaluation about the mathematics chat bot.

In [61]:
!pip3 install mistralai
!pip3 install python-dotenv
!pip3 install mathematics_dataset
!pip3 install pip install sympy==1.6
!pip3 install numpy==1.24.3
!pip3 install transformers==4.48.0
!pip3 install tiktoken==0.8.0
!pip3 install protobuf==5.29.3
!pip3 install sentencepiece==0.2.0
!pip3 install torch


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


I've used the following prompt for the Mistral Large 2.1 model
"Your task is to help solve math problems. Generate three answers that helps solving the problem but doesn't give the solution. When generating the 4th answer to the problem you can give the solution. When you don't know the answer say that you don't know the answer. Write the mathematical part of the answer in latex."

In [2]:
import os
from dotenv import load_dotenv
from mistralai import Mistral

load_dotenv()

api_key = os.environ["MISTRAL_API_KEY"]
agent_id = os.environ["MISTRAL_AGENT_ID"]

client = Mistral(api_key=api_key)

chat_response = client.agents.complete(
    agent_id= agent_id,
    messages=[
        {
            "role": "user",
            "content": """Given the matrix:
                            A = [
                                [4, 1, -2],
                                [1, 3,  0],
                                [-2, 0, 2]
                            ]

                            Tasks:
                            1. Compute the eigenvalues of A.
                            2. Compute the eigenvectors of A for each eigenvalue.
                            3. Verify that for each eigenpair (λ, v), the equation A * v = λ * v holds.
                            4. If possible, diagonalize A by finding matrices P (eigenvectors) and D (diagonal matrix of eigenvalues) such that A = P * D * P⁻¹.
                            """
        },
    ],
)
print(chat_response.choices[0].message.content)

1. To compute the eigenvalues of the matrix $A$, you need to solve the characteristic equation $\det(A - \lambda I) = 0$, where $I$ is the identity matrix. This involves finding the roots of the characteristic polynomial.

2. For each eigenvalue $\lambda$ found in step 1, solve the equation $(A - \lambda I)v = 0$ to find the corresponding eigenvectors $v$. This involves solving a homogeneous system of linear equations for each eigenvalue.

3. To verify that for each eigenpair $(\lambda, v)$, the equation $A * v = \lambda * v$ holds, substitute the eigenvalue $\lambda$ and the corresponding eigenvector $v$ into the equation and check if the left-hand side equals the right-hand side.

4. To diagonalize the matrix $A$, we need to form the matrix $P$ whose columns are the eigenvectors of $A$, and the diagonal matrix $D$ whose diagonal elements are the eigenvalues of $A$. Then, we can check if $A = P * D * P^{-1}$.

   For the given matrix $A = \begin{bmatrix}
   4 & 1 & -2 \\
   1 & 3 & 0 

I have used the [mathematics_dataset](https://github.com/google-deepmind/mathematics_dataset) from Google DeepMind that will work as a test set. The main issue is that I don't know if the Mistral Large model that is used for the mathematics model have been trained on this data. There is a great chans for that. 

Never the less, I made a json file called questions_and_answers of the [mathematics_dataset](https://github.com/google-deepmind/mathematics_dataset) so that I can easily separet the questions from the answers. Ideally I would run all the questions through the mathematics model, but due to budget questions that won't be possible. I'll choose around 50 questions from the dataset. 

In [151]:
import json
import re

def clean_ansi(text):
    """Remove ANSI color codes from text"""
    ansi_pattern = re.compile(r'\x1B(?:[@-Z\\-_]|\\[[0-?]*[ -/]*[@-~])')
    return ansi_pattern.sub('', text)

def process_qa_data(raw_text, output_file="questions_and_answers.json"):
    """Process raw text into question-answer pairs and save to JSON"""
    qa_pairs = []
    
    # Split into lines and process each line
    lines = raw_text.split('\n') if isinstance(raw_text, str) else raw_text
    
    current_section = None
    current_question = []
    previous_answer = None
    
    for line in lines:
        # Skip empty lines
        if not line.strip():
            continue
            
        # Check if this is a section header
        if line.startswith('\u001b[1m') and '\u001b[92m' not in line:
            current_section = clean_ansi(line.strip())
            current_question = []
            previous_answer = None
            continue
            
        # Skip log messages
        if line.startswith('W') and ']' in line:
            continue
        
        # If line contains an answer (marked by green color code)
        if '\u001b[92m' in line:
            parts = line.split('\u001b[92m')
            
            # Get the question part (if any)
            question_part = parts[0].strip()
            
            # If there's a reset code in the question part, it contains a previous answer
            if '\u001b[0m' in question_part:
                q_parts = question_part.split('\u001b[0m')
                if len(q_parts) > 1 and previous_answer is not None:
                    # Complete the previous answer
                    qa_pairs[-1]["answer"] += ' ' + clean_ansi(q_parts[0].strip())
                    # Update the question part to exclude the previous answer
                    question_part = q_parts[1].strip()
            
            # Add any new question part to current question
            if question_part:
                current_question.append(question_part)
            
            # Get the answer
            answer = parts[1].strip()
            if '\u001b[0m' in answer:
                answer = answer.split('\u001b[0m')[0].strip()
            
            # Combine all question parts
            full_question = ' '.join(current_question).strip()
            
            # Add the new QA pair
            if answer:
                qa_pairs.append({
                    "section": clean_ansi(current_section) if current_section else None,
                    "question": full_question,
                    "answer": clean_ansi(answer)
                })
                previous_answer = answer
            
            # Reset question accumulator
            current_question = []
        else:
            # If line doesn't contain an answer, check if it contains a previous answer completion
            if '\u001b[0m' in line:
                parts = line.split('\u001b[0m')
                if previous_answer is not None:
                    # Add to previous answer
                    qa_pairs[-1]["answer"] += ' ' + clean_ansi(parts[0].strip())
                    # Add remaining part to current question
                    if len(parts) > 1 and parts[1].strip():
                        current_question.append(parts[1].strip())
            else:
                # Regular question part
                current_question.append(line.strip())
    
    # Filter out any entries with empty questions
    qa_pairs = [pair for pair in qa_pairs if pair["answer"] and pair["question"]]
    
    # Save to JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(qa_pairs, f, indent=2)
    
    print(f"Output saved to {output_file}")
    return qa_pairs

# Example usage:
if __name__ == "__main__":
    # Get the output from mathematics_dataset.generate
    output = !python -m mathematics_dataset.generate
    
    # Process the output
    qa_data = process_qa_data(output)

Output saved to questions_and_answers.json


In [152]:
import json

def clean_specific_ansi(text):
    """Remove specific ANSI codes"""
    if text:
        return text.replace('\u001b[1m', '').replace('\u001b[0m', '')
    return text

def clean_json_file(input_file="questions_and_answers.json", output_file="clean_qa_data.json"):
    # Read the JSON file
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Clean specific ANSI codes from each field
    clean_data = []
    for item in data:
        clean_item = {
            "section": clean_specific_ansi(item["section"]),
            "question": clean_specific_ansi(item["question"]),
            "answer": clean_specific_ansi(item["answer"])
        }
        clean_data.append(clean_item)
    
    # Write the cleaned data back to a new JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(clean_data, f, indent=2)
    
    print(f"Cleaned data saved to {output_file}")
    return clean_data

# Run the cleaning
clean_data = clean_json_file()

Cleaned data saved to clean_qa_data.json


In [153]:

# Load the JSON data from the file
with open('clean_qa_data.json', 'r') as json_file:
    data = json.load(json_file)

# Extract only the questions
questions = [entry["question"] for entry in data]

# Optionally, you can save the questions to a new JSON file
with open('questions.json', 'w') as json_file:
    json.dump(questions, json_file, indent=4)

Try "google/t5-small-ssm-nq" cannot classify answers correctly when the Mistral Large model gives the output in the form "The answer is 4" and it compares it to only the mathematical answer "4". 

In [3]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from sympy import sympify, Eq, simplify

# Load the T5 model and tokenizer
model_name = "google/t5-small-ssm-nq"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5TokenizerFast.from_pretrained(model_name)

# Example datasets
dataset = [
    {"question": "What is 2 + 2?", "expected_answer": "4", "mistral_answer": "The answer is 4"},
    {"question": "What is 3 * 3?", "expected_answer": "9", "mistral_answer": "When multiplying, 3 * 3 equals 9"},
    {"question": "What is 10 - 3?", "expected_answer": "7", "mistral_answer": "Subtracting, we get 6"}
]

# Function to validate correctness using the model
def validate_correctness(question, mistral_answer, expected_answer):
    def extract_math(expr):
        """
        Extract and simplify the mathematical part of an expression.
        Returns the simplified math expression or None if parsing fails.
        """
        try:
            return simplify(sympify(expr))
        except:
            return None

    # Use T5 to classify the correctness of the Mistral model's output
    prompt = (
        f"Evaluate if the given answer is correct:\n"
        f"Question: {question}\n"
        f"Mistral's Answer: {mistral_answer}\n"
        f"Expected Answer: {expected_answer}\n"
        f"Respond with 'Correct' or 'Incorrect'."
    )

    # Generate T5's judgment
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=10)
    model_judgment = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Also validate mathematically
    mistral_expr = extract_math(mistral_answer)
    expected_expr = extract_math(expected_answer)
    mathematical_correct = mistral_expr == expected_expr

    return {
        "model_judgment": model_judgment,
        "mathematical_correct": mathematical_correct
    }

# Evaluate the dataset
results = []
for item in dataset:
    result = validate_correctness(
        question=item["question"],
        mistral_answer=item["mistral_answer"],
        expected_answer=item["expected_answer"]
    )
    results.append({**item, **result})

# Print results
for res in results:
    print(res)


{'question': 'What is 2 + 2?', 'expected_answer': '4', 'mistral_answer': 'The answer is 4', 'model_judgment': 'incorrect answers', 'mathematical_correct': False}
{'question': 'What is 3 * 3?', 'expected_answer': '9', 'mistral_answer': 'When multiplying, 3 * 3 equals 9', 'model_judgment': 'incorrect answers', 'mathematical_correct': False}
{'question': 'What is 10 - 3?', 'expected_answer': '7', 'mistral_answer': 'Subtracting, we get 6', 'model_judgment': 'incorrect answers', 'mathematical_correct': False}
